In [3]:
import pandas as pd
import glob
import ntpath
import re
from utils import *

data_dir = "D:/Documents/Pierre-Olivier/CEBD1260/Datasets/"

In [5]:
%%time
names = [re.sub('\.csv$', '', ntpath.basename(p)) for p in glob.glob(data_dir + "*.csv")]

dfs = {}
for df_name in names:
    dfs[df_name] = pd.read_csv(f"{data_dir}{df_name}.csv")

dfs["application_train"].shape

Wall time: 28.6 s


(307511, 122)

In [6]:
%%time
bureau_balance = dfs["bureau_balance"].reset_index().groupby(['SK_ID_BUREAU', 'STATUS'])['MONTHS_BALANCE'].aggregate(lambda x: x.max()-x.min()).unstack()
bureau_balance.columns = [f"{bureau_balance.columns.name}_{c}_MONTHS_BALANCE_MAX_MIN_DIFF" for c in bureau_balance.columns]
bureau_balance.reset_index(level=0, inplace=True)
bureau_balance

Wall time: 1min 14s


,SK_ID_BUREAU,STATUS_0_MONTHS_BALANCE_MAX_MIN_DIFF,STATUS_1_MONTHS_BALANCE_MAX_MIN_DIFF,STATUS_2_MONTHS_BALANCE_MAX_MIN_DIFF,STATUS_3_MONTHS_BALANCE_MAX_MIN_DIFF,STATUS_4_MONTHS_BALANCE_MAX_MIN_DIFF,STATUS_5_MONTHS_BALANCE_MAX_MIN_DIFF,STATUS_C_MONTHS_BALANCE_MAX_MIN_DIFF,STATUS_X_MONTHS_BALANCE_MAX_MIN_DIFF
0,5001709,NaN,NaN,NaN,NaN,NaN,NaN,85.0,10.0
1,5001710,5.0,NaN,NaN,NaN,NaN,NaN,47.0,33.0
2,5001711,2.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,5001712,9.0,NaN,NaN,NaN,NaN,NaN,8.0,NaN
4,5001713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0
...,...,...,...,...,...,...,...,...,...
817390,6842884,8.0,NaN,NaN,NaN,NaN,NaN,19.0,27.0
817391,6842885,11.0,NaN,NaN,NaN,NaN,11.0,NaN,NaN
817392,6842886,7.0,NaN,NaN,NaN,NaN,NaN,24.0,NaN
817393,6842887,5.0,NaN,NaN,NaN,NaN,NaN,30.0,NaN


In [7]:
%%time
dfs["bureau"] = dfs["bureau"].merge(bureau_balance,on='SK_ID_BUREAU',how='left')
for f in [f for f in dfs["bureau"].columns if dfs["bureau"][f].isnull().sum()>0]:
    dfs["bureau"][f] = dfs["bureau"][f].fillna(dfs["bureau"][f].median())

Wall time: 1.72 s


In [8]:
%%time
aggr_dicts = {
    "POS_CASH_balance" : {
        'SK_DPD':['max','std','median'],
        'SK_DPD_DEF':['max','std','median'],
    },
    "credit_card_balance" : {
        'AMT_BALANCE':['max','std','median'],
        'AMT_CREDIT_LIMIT_ACTUAL':['max','std','median'],
        'AMT_PAYMENT_CURRENT':['max','std','median'],
        'AMT_RECEIVABLE_PRINCIPAL':['max','std','median'],
        'AMT_RECIVABLE':['max','std','median'],
        'CNT_INSTALMENT_MATURE_CUM':['max','std','median'],
        'SK_DPD':['max','std','median'],
        'SK_DPD_DEF':['max','std','median'],
    },
    "previous_application" : {
        'AMT_ANNUITY':['max','std','median'],
        'AMT_APPLICATION':['max','std','median'],
        'AMT_CREDIT':['max','std','median'],
        'AMT_DOWN_PAYMENT':['max','std','median'],
        'AMT_GOODS_PRICE':['max','std','median'],
        'RATE_DOWN_PAYMENT':['max','std','median'],
        'RATE_INTEREST_PRIMARY':['max','std','median'],
        'RATE_INTEREST_PRIVILEGED':['max','std','median'],
        'DAYS_DECISION':['max','std','median'],
    },
    "installments_payments" : {
        'AMT_INSTALMENT':['max','std','median'],
        'AMT_PAYMENT':['max','std','median'],
    },
    "bureau" : {
        'SK_ID_BUREAU':['nunique'],
        'CREDIT_ACTIVE':['nunique'],
        'CREDIT_CURRENCY':['nunique'],
        'DAYS_CREDIT':[lambda x: (x.max()-x.min(), "max_min_diff")],
        'CREDIT_DAY_OVERDUE':['sum', 'median'],
        'DAYS_CREDIT_ENDDATE':['std'],
        'DAYS_ENDDATE_FACT':['std'],
        'AMT_CREDIT_MAX_OVERDUE':['max'],
        'CNT_CREDIT_PROLONG':['sum', 'median'],
        'AMT_CREDIT_SUM':['sum', 'median'],
        'AMT_CREDIT_SUM_DEBT':['sum', 'median'],
        'AMT_CREDIT_SUM_LIMIT':['sum', 'median'],
        'AMT_CREDIT_SUM_OVERDUE':['sum', 'median'],
        'CREDIT_TYPE':['nunique'],
        'DAYS_CREDIT_UPDATE':['mean'],
        'AMT_ANNUITY':['sum', 'median'],
        'STATUS_0_MONTHS_BALANCE_MAX_MIN_DIFF':['mean'],
        'STATUS_1_MONTHS_BALANCE_MAX_MIN_DIFF':['mean'],
        'STATUS_2_MONTHS_BALANCE_MAX_MIN_DIFF':['mean'],
        'STATUS_3_MONTHS_BALANCE_MAX_MIN_DIFF':['mean'],
        'STATUS_4_MONTHS_BALANCE_MAX_MIN_DIFF':['mean'],
        'STATUS_5_MONTHS_BALANCE_MAX_MIN_DIFF':['mean'],
        'STATUS_C_MONTHS_BALANCE_MAX_MIN_DIFF':['mean'],
        'STATUS_X_MONTHS_BALANCE_MAX_MIN_DIFF':['mean'],
    }
}

main_df = dfs["application_train"]
for table in aggr_dicts:
    main_df = merge_with_aggr(main_df, dfs[table], "SK_ID_CURR", aggr_dicts[table], table)

main_df.shape, main_df.columns.tolist()

Wall time: 27 s


((307511, 216),
 ['SK_ID_CURR',
  'TARGET',
  'NAME_CONTRACT_TYPE',
  'CODE_GENDER',
  'FLAG_OWN_CAR',
  'FLAG_OWN_REALTY',
  'CNT_CHILDREN',
  'AMT_INCOME_TOTAL',
  'AMT_CREDIT',
  'AMT_ANNUITY',
  'AMT_GOODS_PRICE',
  'NAME_TYPE_SUITE',
  'NAME_INCOME_TYPE',
  'NAME_EDUCATION_TYPE',
  'NAME_FAMILY_STATUS',
  'NAME_HOUSING_TYPE',
  'REGION_POPULATION_RELATIVE',
  'DAYS_BIRTH',
  'DAYS_EMPLOYED',
  'DAYS_REGISTRATION',
  'DAYS_ID_PUBLISH',
  'OWN_CAR_AGE',
  'FLAG_MOBIL',
  'FLAG_EMP_PHONE',
  'FLAG_WORK_PHONE',
  'FLAG_CONT_MOBILE',
  'FLAG_PHONE',
  'FLAG_EMAIL',
  'OCCUPATION_TYPE',
  'CNT_FAM_MEMBERS',
  'REGION_RATING_CLIENT',
  'REGION_RATING_CLIENT_W_CITY',
  'WEEKDAY_APPR_PROCESS_START',
  'HOUR_APPR_PROCESS_START',
  'REG_REGION_NOT_LIVE_REGION',
  'REG_REGION_NOT_WORK_REGION',
  'LIVE_REGION_NOT_WORK_REGION',
  'REG_CITY_NOT_LIVE_CITY',
  'REG_CITY_NOT_WORK_CITY',
  'LIVE_CITY_NOT_WORK_CITY',
  'ORGANIZATION_TYPE',
  'EXT_SOURCE_1',
  'EXT_SOURCE_2',
  'EXT_SOURCE_3',
  'APAR

In [18]:
main_df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,bureau_AMT_ANNUITY_sum,bureau_AMT_ANNUITY_median,bureau_STATUS_0_MONTHS_BALANCE_MAX_MIN_DIFF_mean,bureau_STATUS_1_MONTHS_BALANCE_MAX_MIN_DIFF_mean,bureau_STATUS_2_MONTHS_BALANCE_MAX_MIN_DIFF_mean,bureau_STATUS_3_MONTHS_BALANCE_MAX_MIN_DIFF_mean,bureau_STATUS_4_MONTHS_BALANCE_MAX_MIN_DIFF_mean,bureau_STATUS_5_MONTHS_BALANCE_MAX_MIN_DIFF_mean,bureau_STATUS_C_MONTHS_BALANCE_MAX_MIN_DIFF_mean,bureau_STATUS_X_MONTHS_BALANCE_MAX_MIN_DIFF_mean
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0.0,0.0,7.5,3.625,0.0,0.0,0.0,6.0,7.625,5.625
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0.0,0.0,8.0,0.000,0.0,0.0,0.0,6.0,22.000,6.000
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0.0,0.0,8.0,0.000,0.0,0.0,0.0,6.0,22.000,6.000
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0.0,0.0,8.0,0.000,0.0,0.0,0.0,6.0,22.000,6.000


In [15]:
main_df.isnull().sum().sum()/216

87751.04629629629

In [20]:
na_numfiller(main_df, aggregation_func="median")

In [21]:
main_df.isnull().sum().sum()

808391

In [22]:
na_catfiller(main_df)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,bureau_AMT_ANNUITY_sum,bureau_AMT_ANNUITY_median,bureau_STATUS_0_MONTHS_BALANCE_MAX_MIN_DIFF_mean,bureau_STATUS_1_MONTHS_BALANCE_MAX_MIN_DIFF_mean,bureau_STATUS_2_MONTHS_BALANCE_MAX_MIN_DIFF_mean,bureau_STATUS_3_MONTHS_BALANCE_MAX_MIN_DIFF_mean,bureau_STATUS_4_MONTHS_BALANCE_MAX_MIN_DIFF_mean,bureau_STATUS_5_MONTHS_BALANCE_MAX_MIN_DIFF_mean,bureau_STATUS_C_MONTHS_BALANCE_MAX_MIN_DIFF_mean,bureau_STATUS_X_MONTHS_BALANCE_MAX_MIN_DIFF_mean
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0.0,0.0,7.5,3.625,0.0,0.0,0.0,6.0,7.625000,5.625000
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0.0,0.0,8.0,0.000,0.0,0.0,0.0,6.0,22.000000,6.000000
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0.0,0.0,8.0,0.000,0.0,0.0,0.0,6.0,22.000000,6.000000
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0.0,0.0,8.0,0.000,0.0,0.0,0.0,6.0,22.000000,6.000000
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0.0,0.0,8.0,0.000,0.0,0.0,0.0,6.0,22.000000,6.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0.0,0.0,8.0,0.000,0.0,0.0,0.0,6.0,22.000000,6.000000
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0.0,0.0,8.0,0.000,0.0,0.0,0.0,6.0,22.000000,6.000000
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,175108.5,58369.5,12.0,0.000,0.0,0.0,0.0,6.0,19.000000,10.250000
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0.0,0.0,7.0,0.000,0.0,0.0,0.0,6.0,28.000000,6.000000


In [24]:
main_df.isnull().sum().sum()

0